# Gallery Example: M/M/1-PS Queue (Processor Sharing)

This example demonstrates an M/M/1 queue with Processor Sharing scheduling:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Scheduling**: PS (Processor Sharing) - all jobs share the server equally
- **Capacity**: Infinite

Under PS, all jobs in the system receive equal share of the server's capacity, simulating time-sharing systems.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_ps():    """Create M/M/1-PS queueing model"""    model = Network('M/M/1-PS')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.PS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    source.set_arrival(oclass, Exp(1))  # λ = 1    queue.set_service(oclass, Exp(2))   # μ = 2, so ρ = λ/μ = 0.5        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_ps()

## Theoretical Results for M/M/1-PS

For M/M/1-PS with λ=1, μ=2 (ρ=0.5):
- **Utilization**: ρ = 0.5 (same as FCFS)
- **Mean Queue Length**: L = ρ/(1-ρ) = 0.5/(1-0.5) = 1.0 (same as FCFS)
- **Mean Response Time**: W = 1/(μ-λ) = 1/(2-1) = 1.0 seconds (same as FCFS)
- **Throughput**: X = λ = 1.0

**Key Difference**: While mean performance is identical to FCFS, PS provides better perceived performance for short jobs and worse for long jobs.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare PS vs FCFS scheduling
print("\n=== PS vs FCFS Comparison ===")

# Create equivalent FCFS model
def create_mm1_fcfs():
    model_fcfs = Network('M/M/1-FCFS')
    source = Source(model_fcfs, 'mySource')
    queue = Queue(model_fcfs, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model_fcfs, 'mySink')
    
    oclass = OpenClass(model_fcfs, 'myClass')
    source.set_arrival(oclass, Exp(1))
    queue.set_service(oclass, Exp(2))
    
    P = model_fcfs.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_fcfs.link(P)
    
    return model_fcfs

# Solve both models
model_fcfs = create_mm1_fcfs()
solver_ps = MVA(model)
solver_fcfs = MVA(model_fcfs)

avg_table_ps = solver_ps.avg_table()
avg_table_fcfs = solver_fcfs.avg_table()

print("M/M/1-PS Results:")
print(avg_table_ps)

print("\nM/M/1-FCFS Results:")
print(avg_table_fcfs)

print("\nObservation: Mean performance metrics (utilization, queue length, response time)")
print("are identical between PS and FCFS for single-class M/M/1 systems.")
print("The difference lies in the service time variance and job-size fairness.")

In [ ]:
# Analyze PS behavior under different loads
print("\n=== Load Analysis for PS ===")
load_factors = [0.3, 0.5, 0.7, 0.9, 0.95]

print("Load Factor | Utilization | Queue Length | Response Time")
print("-" * 60)

for rho in load_factors:
    # Create model with specified load factor
    model_load = Network(f'M/M/1-PS-rho{rho}')
    source = Source(model_load, 'Source')
    queue = Queue(model_load, 'Queue', SchedStrategy.PS)
    sink = Sink(model_load, 'Sink')
    
    oclass = OpenClass(model_load, 'Class')
    source.set_arrival(oclass, Exp(1))  # λ = 1
    queue.set_service(oclass, Exp(1/rho))  # μ = 1/ρ, so ρ = λ/(1/ρ) = ρ
    
    P = model_load.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_load.link(P)
    
    # Solve and extract metrics
    solver = MVA(model_load)
    avg_table = solver.avg_table()
    
    util = float(avg_table.iloc[1, 1])  # Queue utilization
    queue_length = float(avg_table.iloc[1, 3])  # Queue length
    resp_time = float(avg_table.iloc[1, 2])  # Response time
    
    print(f"    {rho:4.2f}    |    {util:5.3f}    |     {queue_length:5.3f}     |     {resp_time:6.3f}")